In [1]:
import pandas as pd
import os
import re

In [2]:
DATA_PATH = 'C:\\_Repositories\\Kamaz_Control\\data\\train.part1\\train'
PIC_WIDTH = 1920
PIC_HEIGHT = 1020

In [3]:
txt_files = []

In [4]:
# извлечение всех txt-файлов с данными о детектированных деталях
for root, _, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith(".txt"):
             txt_files.append(os.path.join(root, file))

In [5]:
# таблица с информацией о деталях
columns = ['detail', 'number', 'x1', 'y1', 'x2', 'y2', 'width', 'height', 'part_width', 'part_height', 'part_square']
details_table = pd.DataFrame(columns=columns)

In [6]:
def prune_float(numb):
    return float('{:.4f}'.format(numb))

In [7]:
# составление таблицы
for path_to_txt in txt_files:
    txt_name = os.path.splitext(os.path.basename(path_to_txt))[0]
    detail = re.sub('\d', '', txt_name)
    number = int(re.sub('[^0-9]', '', txt_name))
    with open(path_to_txt, 'r') as f:
        f.readline()
        data = f.readline()
        x1, y1, x2, y2 = map(int, data.split())
    width, height = x2-x1, y2-y1
    part_width, part_height = prune_float(width/PIC_WIDTH), prune_float(height/PIC_HEIGHT)
    part_square = prune_float(part_width*part_height)
    details_table.loc[len(details_table)]=[detail, number, x1, y1, x2, y2, width, height, part_width, part_height, part_square]

In [8]:
details_table.head(11)

,detail,number,x1,y1,x2,y2,width,height,part_width,part_height,part_square
0,kol,1,801,488,1182,849,381,361,0.1984,0.3539,0.0702
1,kol,10,556,212,874,588,318,376,0.1656,0.3686,0.0610
2,kol,2,704,269,973,690,269,421,0.1401,0.4127,0.0578
3,kol,3,762,138,1202,556,440,418,0.2292,0.4098,0.0939
4,kol,4,539,136,1040,615,501,479,0.2609,0.4696,0.1225
5,kol,5,681,8,1167,446,486,438,0.2531,0.4294,0.1087
6,kol,6,747,30,1207,465,460,435,0.2396,0.4265,0.1022
7,kol,7,605,337,1041,741,436,404,0.2271,0.3961,0.0900
8,kol,8,818,227,1116,515,298,288,0.1552,0.2824,0.0438
9,kol,9,716,254,1154,638,438,384,0.2281,0.3765,0.0859


In [9]:
details_table.shape

(100, 11)

In [10]:
details_table.to_csv('details_extimation.csv')

In [11]:
details_table.groupby('detail').size()

detail
Ekranturb          10
Kkomp              10
Perehodnik         10
Pod                10
Turb               10
kol                10
krysh              10
turbokompressor    10
usel               10
val                10
dtype: int64

In [12]:
details_size = details_table[['detail', 'width', 'height']].groupby('detail').agg(['min', 'max'])
details_size

width       height     
                  min   max    min  max
detail                                 
Ekranturb         144   597    359  588
Kkomp             284   983    331  943
Perehodnik        209   563    130  430
Pod               465   749    452  699
Turb              348   945    396  707
kol               269   501    288  479
krysh             330   852    422  679
turbokompressor   575   874    554  803
usel              713  1074    453  674
val               880  1213    350  531

In [13]:
details_table[['detail', 'width', 'height']].groupby('detail').agg(['min', 'max']).to_csv('details_size.csv')